In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
url = 'FB.csv'
meta = pd.read_csv(url)
meta = meta.dropna(how='any',axis=0) #If there is missing data
meta['Date'].apply(pd.to_datetime) #For ease of adjusting dates later on
len(meta.index)

2382

In [3]:
meta = meta[['Close'] + [col for col in meta if col not in ['Adj Close', 'Close']] + ['Adj Close']]

In [4]:
meta = meta.sort_values('Date')
meta.head()

,Close,Date,Open,High,Low,Volume,Adj Close
0,38.230000,2012-05-18,42.049999,45.000000,38.000000,573576400,38.230000
1,34.029999,2012-05-21,36.529999,36.660000,33.000000,168192700,34.029999
2,31.000000,2012-05-22,32.610001,33.590000,30.940001,101786600,31.000000
3,32.000000,2012-05-23,31.370001,32.500000,31.360001,73600000,32.000000
4,33.029999,2012-05-24,32.950001,33.209999,31.770000,50237200,33.029999


In [5]:
#Method 1

In [6]:
#RNN model
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, num_inputs, hidden_dims, layer_dims, num_outputs):
        super(RNN, self).__init__()
        self.hidden_dims = hidden_dims #Numer of features in the hidden state
        self.layer_dims = layer_dims #Number of recurrent layers
        self.rnn = nn.RNN(num_inputs, hidden_dim, layer_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, num_outputs)
        
    def forward(self, X):
        h_0 = torch.zeros(self.layer_dim, X.size(0), self.hidden_dim).requires_grad_()
        out, h_0 = self.rnn(X, h_0.detach())
        out = out[:,-1,:]
        out.self.fc(out)
        return out

In [7]:
#RNN model
num_input, hidden_dim, layer_dim, num_output = 14, 15, 2, 1
rnn_model = RNN(num_input, hidden_dim, layer_dim, num_output)

In [8]:
#Sliding window data
def sliding_window(data, timeframe):
    numpydata = data.to_numpy() 
    data = []

    # create all possible sequences of length seq_len
    for index in range(len(numpydata) - timeframe): 
        data.append(numpydata[index: index + timeframe])

    return np.array(data)

sw_meta = sliding_window(meta.iloc[:,0:1], 15)

In [9]:
meta_data = pd.DataFrame(sw_meta.T.reshape(-1, 15))
meta_data.columns = ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7', 'Day 8', 'Day 9', 'Day 10', 'Day 11', 'Day 12', 'Day 13', 'Day 14', 'Target']
meta_data = meta_data[['Target'] + [col for col in meta_data.columns if col != 'Target' ]]

In [10]:
from sklearn.model_selection import train_test_split

def feature_label_split(dataframe, target):
    y = dataframe.iloc[:,0:1]
    X = dataframe.iloc[:,1:]
    return X, y

def train_val_test_split(dataframe, target, test_ratio):
    val_ratio = test_ratio / (1 - test_ratio)
    X, y = feature_label_split(dataframe, target)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, shuffle = False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = val_ratio, shuffle = False)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [11]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(meta_data, 'Target', 0.2)

In [12]:
#Scaling datasets
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_array = scaler.fit_transform(X_train)
X_val_array = scaler.transform(X_val)
X_test_array = scaler.transform(X_test)

y_train_array = scaler.fit_transform(y_train)
y_val_array = scaler.transform(y_val)
y_test_array = scaler.transform(y_test)

In [13]:
import torch
from torch.utils.data import TensorDataset

train_feat = torch.Tensor(X_train_array)
train_targ = torch.Tensor(y_train_array)
val_feat = torch.Tensor(X_val_array)
val_targ = torch.Tensor(y_val_array)
test_feat = torch.Tensor(X_test_array)
test_targ = torch.Tensor(y_test_array)

training_set = TensorDataset(train_feat, train_targ)
valid_set = TensorDataset(val_feat, val_targ)
test_set = TensorDataset(test_feat, test_targ)

In [14]:
from torch.utils.data import DataLoader

batch_size = 50
train_dl = DataLoader(training_set, batch_size =  batch_size, drop_last = True)
val_dl = DataLoader(valid_set, batch_size =  batch_size, drop_last = True)
test_dl = DataLoader(test_set, batch_size =  batch_size, drop_last = True)

In [15]:
#Optimiser
class Optimization():
    def __init__(self, model, criterion, updater):
        self.model = model
        self.criterion = criterion
        self.updater = updater
        self.train_losses = []
        self.valid_losses = []
        
        def training_step(self, inputs, labels):
            self.model.train()
            y_hat = self.model(inputs)
            loss = self.criterion(labels, y_hat)
            loss.backward()
            self.updater.step()
            self.updater.zero_grad()
            return loss.item()

In [16]:
#Training the model
def RNN_model_trainer(self, train_dataloader, val_dataloader, batch_size, num_epochs, num_features):
    for epoch in range(1, num_epochs + 1):
        batch_loss = []
        for train_inputs, train_labels in train_loader:
            train_inputs = train_inputs.view([batch_size, -1, num_features]).to(device)
            train_labels = train_labels.to(device)
            loss = self.training_step(train_inputs, train_labels)
            batch_loss.append(loss)
            training_loss = np.mean(batch_loss)
            self.train_losses.append(training_loss)
            
            with torch.no_grad():
                batch_val_loss = []
                for val_inputs, val_labels in val_loader:
                    val_inputs = val_inputs.view([batch_size, -1, num_features]).to(device)
                    val_labels = val_labels.to(device)
                    self.model.eval()
                    y_hat = self.model(val_inputs)
                    val_loss = criterion(val_labels, y_hat).item()
                    batch_val_loss.append(val_loss)
                validation_loss = np.mean(batch_val_loss)
                self.valid_losses.append(validation_loss)

In [17]:
lr, num_epochs = 0.2, 60
criterion_3 = nn.MSELoss()
updater_3 = torch.optim.SGD(rnn_model.parameters(), lr = lr)

In [18]:
from datetime import datetime

opt = Optimization(model = rnn_model, criterion = criterion_3, updater = updater_3)
opt.RNN_model_trainer(train_dataloader = train_dl, val_dataloader = val_dl, batch_size = 50, num_epochs = num_epochs, num_features = num_input)

AttributeError: 'Optimization' object has no attribute 'RNN_model_trainer'

In [ ]:
#Method 2